# Opdracht classificatie 1 - logistic regression

Logistische regressie is een bijzonder populaire classificatietechniek. Enerzijds door zijn eenvoud en relatief lage eisen die het stelt in termen van rekenkracht. In veel gevallen is de performantie qua accuracy vergelijkbaar (en soms beter) dan gecompliceerdere algoritmes zoals de support vector machines.
Daarnaast heeft logistische regressie het voordeel dat het getrainde model een voorspelling doet in termen van de kans dat de input tot een bepaalde klasse behoort. Uit deze kans kan je afleiden hoe overtuigd het model is van de gemaakte voorspelling.

Het is de bedoeling om via enkele classificatieopdrachten inzicht te verkrijgen in:
- Correct trainen en het uitvoeren van hyperparameter tuning bij logistische regressie.
- Classificaties kunnen uitvoeren via logistische regressie.
- Feature engineering uitvoeren.
- Interpreteren van de verschillende performance metrics: accuracy, recall, precision, f1-score, ROC.
- Kunnen omgaan met niet-gebalanceerde data en het kunnen regelen tussen het aantal false positives en false negatives. 
- Weten wanneer je te maken hebt met overfitting en underfitting en de juiste bijstellingen kunnen doen.


In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import missingno as msno
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score

## Opdracht 1: Sensors data


Het bestand sensors_data_test is een tijdelijke bestand om onze sensorsdata te simuleren
Train nu een logic regression model dat op basis van de features een zo goed mogelijke predictie kan doen of er een fles is genomen.

### Kolom toevoegen


### Inlezen van de dataset en vooranalyse

In [2]:
# inlezen van de dataset
dataset = pd.read_csv('sensors_data_test_project.csv')
dataset.head(10)

,RackId,RackRow,WeightPre,WeightPost,WeightDiff,DistMin,DistMax,DistAvg,DistTime,DayTime
0,3,1,60,50,2,1,40,10,3,12/25/1924
1,2,2,38,80,3,1,40,19,4,6:00
2,3,1,40,50,1,1,40,23,4,6:30
3,2,2,88,80,5,1,1,40,6,7:00
4,1,2,65,80,1,1,40,28,2,7:01
5,2,1,81,80,0,1,40,35,1,7:02
6,2,2,65,80,1,1,40,20,2,7:03
7,2,2,65,80,1,40,20,3,10,7:04
8,3,2,65,40,2,1,1,20,3,7:05
9,3,2,60,80,0,1,1,20,4,7:06


Controleer of de dataset inconsistenties of foute waarden bevat. Gebruik listwise deletion. Dit betekent dat je alle gegevens van een persoon uit de dataset verwijdert van zodra er 1 feature foutief is of ontbreekt.

In [3]:
# inconsistenties opsporen

#dataset.info
# RackId moet groter zijn dan 0
dataset= dataset[dataset['RackId'] > 0]
# RackRow moet groter zijn dan 0
dataset= dataset[dataset['RackRow'] > 0]


In [4]:
#(rij met) NaN value verwijderen
#Inplace=True om wijziging 'op te slaan'
dataset.dropna(inplace=True) 
dataset.head(10)

,RackId,RackRow,WeightPre,WeightPost,WeightDiff,DistMin,DistMax,DistAvg,DistTime,DayTime
0,3,1,60,50,2,1,40,10,3,12/25/1924
1,2,2,38,80,3,1,40,19,4,6:00
2,3,1,40,50,1,1,40,23,4,6:30
3,2,2,88,80,5,1,1,40,6,7:00
4,1,2,65,80,1,1,40,28,2,7:01
5,2,1,81,80,0,1,40,35,1,7:02
6,2,2,65,80,1,1,40,20,2,7:03
7,2,2,65,80,1,40,20,3,10,7:04
8,3,2,65,40,2,1,1,20,3,7:05
9,3,2,60,80,0,1,1,20,4,7:06


In [5]:
#PLANNING


#Classifier (Volle fles in, volle fles uit, lege fles in, lege fles uit, onveranderd)

# Toevoegen van 'Fles' kolom op basis van DistTime
#dataset.insert(0,'Fles', dataset['DistTime'])

#is de postweight groter dan de preweight? zoja dan is het een volle fles
#dataset['Post_groter_dan_pre'] = dataset['WeightPost'].gt(dataset['WeightPre'])

##random nieuw kolom toevoegen
#dataset['new_column_name'] = 5



# dataset.insert(0,'Fles_vol',0)
#dataset.insert(0,'Fles_vol',0)


# dataset['Post_groter_dan_pre'] = dataset['WeightPost'].gt(dataset['WeightPre'])
# # #volle fles op basis vanpost groter dan pre
# dataset.insert(0,'Fles_vol',dataset['Post_groter_dan_pre'])

# # True en False  van Fles_volomzetten naar 0 en 1:
# dataset.Fles_vol.replace({False:0,True:1},inplace=True)



dataset.drop('DayTime',axis=1,inplace=True)
dataset.drop('RackId',axis=1,inplace=True)
dataset.drop('RackRow',axis=1,inplace=True)


#(volle) fles nemen
# dataset['Fles_vol'].values[dataset['DistTime'] <=  3 &  (dataset['WeightDiff'] > 3 )] = 1
# #(volle) fles leggen
# dataset['Fles_vol'].values[dataset['DistTime'] > 3 &  (dataset['WeightDiff'] > 3 )] = 0





# dataset['Fles_vol_leeg'].values[dataset['WeightDiff'] <=  3 ] = 0
# # #(volle) fles leggen
# #dataset['Fles_vol'].values[dataset['WeightDiff'] > 3] = 0

# #lege fles nemen
# dataset['Fles_vol_leeg'].values[dataset['DistTime'] <=  2 & (dataset['WeightDiff'] < 1 )] = 1


# #kans dat een fles vol is
# dataset.insert(0,'Fles_vol',0)
# dataset['Fles_vol'].values[dataset['WeightDiff'] < 2] = 0
# #volle fles
# dataset['Fles_vol'].values[dataset['WeightDiff'] > 2] = 1



#FLES
dataset['Post_groter_dan_pre'] = dataset['WeightPost'].gt(dataset['WeightPre'])
# fles in  op basis vanpost groter dan pre
dataset.insert(0,'Bottle_in',dataset['Post_groter_dan_pre'])
dataset.drop('Post_groter_dan_pre',axis=1,inplace=True)


dataset['Pre_groter_dan_post'] = dataset['WeightPre'].gt(dataset['WeightPost'])
#  fles uit op basis van pre  groter dan post
dataset.insert(0,'Bottle_out',dataset['Pre_groter_dan_post'])
dataset.drop('Pre_groter_dan_post',axis=1,inplace=True)

# True en False  van Fles_volomzetten naar 0 en 1:
dataset.Bottle_in.replace({False:0,True:1},inplace=True)

dataset.Bottle_out.replace({False:0,True:1},inplace=True)


#Volle fles in
dataset.insert(0,'Bottle_full',dataset['Bottle_in'])
dataset['Bottle_full'].values[dataset['WeightDiff'] < 2] = 0
dataset['Bottle_full'].values[dataset['WeightDiff'] > 2] = 1

#bottle full in
dataset.insert(0,'Bottle_full_in',dataset['Bottle_in'])
dataset['Bottle_full_in'].values[dataset['Bottle_full'] == 1 & (dataset['Bottle_in'] == 1 ) ] = 1
dataset['Bottle_full_in'].values[dataset['Bottle_in'] == 1 & (dataset['Bottle_full'] == 1 ) ] = 1
dataset['Bottle_full_in'].values[dataset['Bottle_full'] == 0 & (dataset['Bottle_in'] == 1 ) ] = 0
dataset['Bottle_full_in'].values[dataset['Bottle_in'] == 0 & (dataset['Bottle_full'] == 1 ) ] = 0

#bottle full out
dataset.insert(0,'Bottle_full_out',dataset['Bottle_out'])
dataset['Bottle_full_out'].values[dataset['Bottle_full'] == 1 & (dataset['Bottle_out'] == 1 ) ] = 1
dataset['Bottle_full_out'].values[dataset['Bottle_out'] == 1 & (dataset['Bottle_full'] == 1 ) ] = 1
dataset['Bottle_full_out'].values[dataset['Bottle_full'] == 0 & (dataset['Bottle_out'] == 1 ) ] = 0
dataset['Bottle_full_out'].values[dataset['Bottle_out'] == 0 & (dataset['Bottle_full'] == 1 ) ] = 0




#Bottle
dataset.insert(0,'Bottle',0)
dataset['Bottle'].values[dataset['Bottle_full_in'] == 1] = 1
dataset['Bottle'].values[dataset['Bottle_full_out'] == 1] = 2

    

dataset.head(20)



,Bottle,Bottle_full_out,Bottle_full_in,Bottle_full,Bottle_out,Bottle_in,WeightPre,WeightPost,WeightDiff,DistMin,DistMax,DistAvg,DistTime
0,0,0,0,0,1,0,60,50,2,1,40,10,3
1,1,0,1,1,0,1,38,80,3,1,40,19,4
2,0,0,0,0,0,1,40,50,1,1,40,23,4
3,2,1,0,1,1,0,88,80,5,1,1,40,6
4,0,0,0,0,0,1,65,80,1,1,40,28,2
5,0,0,0,0,1,0,81,80,0,1,40,35,1
6,0,0,0,0,0,1,65,80,1,1,40,20,2
7,0,0,0,0,0,1,65,80,1,40,20,3,10
8,0,0,0,0,1,0,65,40,2,1,1,20,3
9,0,0,0,0,0,1,60,80,0,1,1,20,4


Bepaal de verdeling van het gewichtsverschil. Is de dataset gebalanceerd?

In [6]:
# gebalanceerdheid controleren
sns.countplot(data=dataset, x="Fles_vol")
plt.title('Countplot Fles_vol')


ValueError: Could not interpret input 'Fles_vol'

Bepaal de statistische kerngetallen van de verschillende features en target. Gebruik hiervoor de *describe* functie.

In [ ]:
# Lineaire correlatie tussen de features - heatmap

dataset.corr()

# Visualiseer de onderlinge correlatiecoëfficiënten
f, ax = plt.subplots(figsize=(12, 10))
corr = dataset.corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),
            square=True, ax=ax,annot=True)

In [ ]:
#sns.histplot(data=dataset, x='quality')
sns.histplot(dataset['Fles_vol'],bins=10,kde=False,color='lightgreen')
plt.title('Histogram Fles')

### Preprocessing en opsplitsen van de dataset

Splits de dataset in **features en targets**.

In [ ]:
# Opsplitsen in features en targets
y = dataset['Fles_vol'].values
X = dataset.drop(['Fles_vol'], axis = 1).values

print(X.shape)
print(y.shape)


#1 lijst met 303 waarden, 13 features

Creëer een **trainingset en een testset**. Zorg dat er 100 patiënten in de testset steken. 

In [ ]:
# Opsplitsen/opdelen in training set en test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

#training set
print(X_train.shape)

#test set
print(X_test.shape)



**Schaal de dataset**. Zorg er dus voor dat de features op een gelijke schaalverdeling staan. Voor het scalen kan gebruik gemaakt worden van de *preprocessing.StandardScaler()*. Meer info over het gebruik ervan is te vinden op http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

In [ ]:
# Normaliseren / scaling van de training set en de test set
scaler = StandardScaler() #scalen
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train[1]

### Trainen van een logistic regression classifier en testen van het bekomen model

Train een logistic regression classifier op de training data. Kies C=1 als startwaarde. Mocht de dataset niet gebalanceerd zijn (de ene klasse komt frequenter voor dan de andere klasse) dan kan je bij de creatie van het logistic regression model de parameter class_weight='balanced' meegeven. Meer info: http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

Test het getrainde model op de test set. Bepaal hierbij de confusion matrix, de accuracy en het classification report. Wat zijn de bevindingen? Probeer ook verschillende solvers uit. Bepaal ook telkens de ROC en formuleer conclusies.

Probeer de performantie van de classifier te verhogen door de parameter C te veranderen.

Evalueer de modellen telkens via de confusion matrix, classification report en indien mogelijk via de ROC-curve. Bespreek de resultaten.

In [ ]:
# Train een logistic regression classifier

logreg = linear_model.LogisticRegression(C=1e3, solver='lbfgs') # C= Inverse of regularization strength;
                                                # must be a positive float. Like in support vector machines,



    # smaller values specify stronger regularization.
logreg.fit(X, y)

print('coefficiënten: ',logreg.coef_)
print('intercept:',logreg.intercept_)

In [ ]:
# testen van de logistic regression classifier

y_pred = logreg.predict(X_test)

print(classification_report(y_test,y_pred))

print(accuracy_score(y_test,y_pred)*100)


print(confusion_matrix(y_test,y_pred))

In [ ]:
# ROC

import sklearn.metrics as metrics
# calculate the fpr and tpr for all thresholds of the classification
probs = logreg.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#probability prediciton van ROC
print(probs)

### Feature engineering
Creeër hogere orde features door gebruik te maken van *preprocessing.PolynomialFeatures*. Meer info is te vinden op http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html

Deze functie zal automatische hogere orde features aanmaken door het combineren van de aanwezige features. Heb je bijvoorbeeld drie features, zijnde A, B en C dan worden bij de keuze van een derde orde PolynomialFeatures volgende nieuwe features bijgemaakt:
$A^3, B^3,C^3,A^2B,A^2C,AB^2, B^2C,...$

Experimenteer met verschillende ordes en gebruik de regularisatieparameter C om de performantie te verhogen. Voor indien nodig ook regularisatie uit via een L1 of L2 penalty.

**Opgepast**: het kiezen van een te hoge orde zorgt voor een exponentiële toename aan features waardoor de logistic regression classifier niet meer binnen aanvaardbare tijd getraind kan worden. Advies is om niet hoger te gaan dan 4de orde.

In [ ]:
# Toevoegen van extra features
# Modeloptimalisatie en Hyperparameter tuning
# Automatisch toevoegen van hogere orde features

from sklearn.preprocessing import PolynomialFeatures


# Aanmaken van de hogere orde features
graad = 3

poly = PolynomialFeatures(graad)
poly.fit(X_train)
X_train_poly = poly.transform(X_train)
X_test_poly = poly.transform(X_test)
print('dimensie van X_train_poly: ',X_train_poly.shape)
print('dimensie van X_test_poly: ',X_test_poly.shape)

#scalen (optioneel?)
scaler_poly = StandardScaler()
scaler_poly.fit(X_train_poly)

X_train_poly = scaler_poly.transform(X_train_poly)
X_test_poly = scaler_poly.transform(X_test_poly)


# met L2 regularisatie via Ridge regression
lregmodel_poly = Ridge(alpha=20,tol=0.0001,fit_intercept=True)
lregmodel_poly.fit(X_train_poly,y_train)

print('R2 score op test set via L2: ',lregmodel_poly.score(X_test_poly,y_test))
# R2 -score via L2 op de trainingset
print('R2 score op training set via L2: ',lregmodel_poly.score(X_train_poly,y_train))


# met L1 regularisatie via Lasso regression
lregmodel_poly = Lasso(alpha=0.001,tol=0.001,fit_intercept=True)
lregmodel_poly.fit(X_train_poly,y_train)      
  
      
print('R2 score op test set via L1: ',lregmodel_poly.score(X_test_poly,y_test))
  
# R2 -score via L1 op de trainingset
print('R2 score op training set via L1: ',lregmodel_poly.score(X_train_poly,y_train))  


Wat zijn de bevindingen? Formuleer een conclusie. Bespreek hierin de performantie van de getrainde modellen. Wat is de invloed van de parameter C en van het aantal features? Heb je te maken gehad met underfitting en overfitting en hoe heb je dit bepaald? Welke accuracy werd bekomen en hoe zit het met de Recall en Precision? Is de grootte van de trainingset voldoende?

Antwoord: Er was een schommeling van 0.02 bij accuracy afhankelijk van hoe ik de C-waarde wijzigde (C1e5->C1e1), mijn accuracy is 80.0, ook de recall en precision wijzigde mee met de C-waarde (schommeling van 0.1)

Bij het toevoegen van hogere orde features heb ik de R2 score bij de trainingsets op 0.96 en 0.98.

### Voorspel aan de hand van het getrainde model

Voorspel of iemand met onderstaande medische parameters als dan niet hartpatient is. Geef ook de zekerheid van het model weer (kansen dat de patiënt tot een bepaalde klasse behoort).

age: 37 - sex: man, cp:3, trestbps: 140, chol:274, fbs: 1, restecg: 1, thalach: 153, exang: 0, oldpeak: 1.4, 
slope: 2, ca: 1, thal: 3


In [ ]:
# Voorspelling van de target.
target = np.array([[37,0,3,140,274,1,1,153,0,1.4,2,1,3]])

#scaling
scaler.transform(target)

print(logreg.predict(target))

print(logreg.predict_proba(np.array([37,0,3,140,274,1,1,153,0,1.4,2,1,3]).reshape(1,-1)))






Standaard zal het model een sample toewijzen aan klasse 1 van zodra de probiliteit boven de threshold van 50% uisteekt. 
Men wil echter de kans op false negatives drastisch verminderen door het aanpassen van de threshold. Welke threshold moet men instellen om ervoor te zorgen dat het model op de test set geen false negatives meer voorspelt en toch nog een zo hoog mogelijke accuraatheid heeft?
Stel het aantal false negatives in functie van de threshold grafisch voor. Bespreek de resultaten.

In [ ]:
# reduceren van het aantal false negatives door de threshold aan te passen
#threshold verlagen!



## Opdracht 2. Penguins


De dataset `penguins.csv` bevat data van verschillende soorten pinguïns. Deze dataset is origineel bedoeld om te kijken of het mogelijk is om te voorspellen tot welke soort een pinguïns behoort aan de hand van de andere variabelen.

<img src='penguins.jpg'>

<br />

| Variable | Beschrijving |
| --- | --- | 
|studyName 	|De studie waar de data van de penguin oorspronkelijk vandaan komt. |
|Sample Number|	Een unieke ID die iedere penguin die in het onderzoek voorkomt krijgt.
|Species |	 De soorten penguins die voorkomen in de dataset.|
|Island | Het eiland waar de penguin geobserveerd werd.|
|Clutch Completion | Of er minstens 1 eitje uit het nest uitgekomen is.|
|Culmen Length (mm) |	De lengte van de snavel in millimeter. |
|Culmen Depth (mm) | De diepte van de snavel in millimeter. |
|Flipper Length (mm)  	| De lengte van de vin in millimeter. |
|Body Mass (g) | Het gewicht van de penguin in gram. |
|Sex | Het geslacht (binair: 'FEMALE' - vrouw or 'MALE' - man)|
|Comments | Specifieke commentaar die bij een bepaalde penguin hoort.|

<br />   
lees de dataset in en toon de eerste 7 rijen.  <br />   
De kolom comments heeft amper data, deze kolom mag uit de dataset verwijdert worden.  <br />   
Gebruik listwise deletion bij de overige ontbrekende waarden. Is de dataset gebalanceerd? Maw, zijn er evenveel penguins per soort?  <br />   
Bouw een logistic regression model dat zo accuraat mogelijk de penguinsoort kan bepalen. Hierbij pas je alle overige preprocessing stappen toe die je nodig acht. Het is ook toegestaann om features bij te maken of weg te laten.
<br />   
Schrijf jouw conclusies op met betrekking tot het finaal bekomen model.


In [ ]:
dataset = pd.read_csv('sensors_data_test.csv')
print('Dimensie van de dataset:',dataset.shape)
dataset.head(7)

In [ ]:
# uitwerking van de opdracht: 

#comments kolom verwijderen
dataset.drop('DayTime;;;;',axis=1,inplace=True)

dataset.insert(0,'Fles',0)
#lege fles
dataset['Fles'].values[dataset['WeightDiff'] < 2] = 1
#volle fles
dataset['Fles'].values[dataset['WeightDiff'] > 2] = 2
dataset['Fles'].values[dataset['DistTime'] > 5] = 3
dataset['Fles'].values[dataset['DistTime'] < 5] = 4




dataset.head(7)

In [ ]:
#(rij met) NaN value verwijderen
#Inplace=True om wijziging 'op te slaan'
dataset.dropna(inplace=True) 

dataset.head(20)

In [ ]:
#One-hot-encoding toepassen omdat ik strings moet omzetten naar getallen

#Species omzetten
dataset = pd.concat([dataset,pd.get_dummies(dataset['Species'], prefix='Species')],axis=1)
dataset.drop(['Species'],axis=1, inplace=True)

#Island  omzetten
dataset = pd.concat([dataset,pd.get_dummies(dataset['Island'], prefix='Island')],axis=1)
dataset.drop(['Island'],axis=1, inplace=True)

#Clutch completion  omzetten
dataset = pd.concat([dataset,pd.get_dummies(dataset['Clutch Completion'], prefix='Clutch Completion')],axis=1)
dataset.drop(['Clutch Completion'],axis=1, inplace=True)



#Sex/geslacht tabel omzetten
dataset = pd.concat([dataset,pd.get_dummies(dataset['Sex'], prefix='Sex')],axis=1)
dataset.drop(['Sex'],axis=1, inplace=True)

dataset.head()

In [ ]:
# gebalanceerdheid controleren
sns.countplot(data=dataset, x="Flipper Length (mm)")
plt.title('Countplot Flipper Length (mm)')

In [ ]:
# Opsporen van ontbrekende waarden

missing_values_count = dataset.isnull().sum()
print(missing_values_count)
msno.matrix(dataset)

In [ ]:
# Lineaire correlatie tussen de features - heatmap

dataset.corr()

# Visualiseer de onderlinge correlatiecoëfficiënten
f, ax = plt.subplots(figsize=(10, 8))
corr = dataset.corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),
            square=True, ax=ax,annot=True)

In [ ]:
# Opsplitsen in features en targets
y = dataset['Flipper Length (mm)'].values
X = dataset.drop(['Flipper Length (mm)'], axis = 1).values

print(X.shape)
print(y.shape)



#1 lijst met 333 waarden, 10 features

In [ ]:
# Opsplitsen/opdelen in training set en test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

#training set
print(X_train.shape)

#test set
print(X_test.shape)

In [ ]:
# Normaliseren / scaling van de training set en de test set

# Om conversion warning te vermijden:
X_train = X_train.astype('float64')
X_test = X_test.astype('float64')


scaler = StandardScaler() #scalen
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train[1]

In [ ]:
# Train een logistic regression classifier

logreg = linear_model.LogisticRegression(C=1e3, solver='lbfgs') # C= Inverse of regularization strength;
                                                # must be a positive float. Like in support vector machines,
                                                # smaller values specify stronger regularization.
logreg.fit(X, y)

print('coefficiënten: ',logreg.coef_)
print('intercept:',logreg.intercept_)

In [ ]:
# testen van de logistic regression classifier

y_pred = logreg.predict(X_test)

print(classification_report(y_test,y_pred))

print(accuracy_score(y_test,y_pred)*100)

print(confusion_matrix(y_test,y_pred))

In [ ]:
# ROC

import sklearn.metrics as metrics
# calculate the fpr and tpr for all thresholds of the classification
probs = logreg.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#probability prediciton van ROC
print(probs)

In [ ]:
# Toevoegen van extra features
# Modeloptimalisatie en Hyperparameter tuning
# Automatisch toevoegen van hogere orde features

from sklearn.preprocessing import PolynomialFeatures


# Aanmaken van de hogere orde features
graad = 4

poly = PolynomialFeatures(graad)
poly.fit(X_train)
X_train_poly = poly.transform(X_train)
X_test_poly = poly.transform(X_test)
print('dimensie van X_train_poly: ',X_train_poly.shape)
print('dimensie van X_test_poly: ',X_test_poly.shape)

#scalen (optioneel?)
scaler_poly = StandardScaler()
scaler_poly.fit(X_train_poly)

X_train_poly = scaler_poly.transform(X_train_poly)
X_test_poly = scaler_poly.transform(X_test_poly)


# met L2 regularisatie via Ridge regression
lregmodel_poly = Ridge(alpha=20,tol=0.01,fit_intercept=True)
lregmodel_poly.fit(X_train_poly,y_train)

print('R2 score op test set via L2: ',lregmodel_poly.score(X_test_poly,y_test))
# R2 -score via L2 op de trainingset
print('R2 score op training set via L2: ',lregmodel_poly.score(X_train_poly,y_train))


# met L1 regularisatie via Lasso regression
lregmodel_poly = Lasso(alpha=0.1,tol=0.01,fit_intercept=True)
lregmodel_poly.fit(X_train_poly,y_train)      
  
      
print('R2 score op test set via L1: ',lregmodel_poly.score(X_test_poly,y_test))
  
# R2 -score via L1 op de trainingset
print('R2 score op training set via L1: ',lregmodel_poly.score(X_train_poly,y_train)) 

In [ ]:
#R2- score op trainingsets: 0.93 en 0.90

## Opdracht 3.  Star Wars

Train een logistic regression classifier aan de hand van de dataset 'StarWars.csv') om te voorspellen of iemand al dan niet een Star Wars fan is.
Gebruik one-hot encondig waar nodig. Zorg ervoor dat de test set uit minstens 200 samples bestaat.
Evalueer de getrainde classifier via de geziene metrics en bespreek de resultaten.

In [ ]:
dataset = pd.read_csv('StarWars.csv')
dataset.head(35)

# we merken op dat rij 19 en 24 allemaal nullen bevat

In [ ]:
#dataset.info()

In [ ]:
# Visualiseer de onderlinge correlatiecoëfficiënten
f, ax = plt.subplots(figsize=(10, 8))
corr = dataset.corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),
           square=True, ax=ax,annot=True)

In [ ]:
#one hot encoding! (for loop?)

#Have you seen any of the 6 films in the Star Wars franchise? kolom verwijderen en omzetten naar ..yes and ..no

#omzetten/features toevoegen
dataset = pd.concat([dataset,pd.get_dummies(dataset['Have you seen any of the 6 films in the Star Wars franchise?'], prefix='Have you seen any of the 6 films in the Star Wars franchise?')],axis=1)
#originele kolom verwijderen
dataset.drop(['Have you seen any of the 6 films in the Star Wars franchise?'],axis=1, inplace=True)

dataset.head(25)

In [ ]:
#we verwijderen rij 19 en 24  van kolom have you..No omdat ze daar 1 zijn
dataset.drop(dataset.loc[dataset['Have you seen any of the 6 films in the Star Wars franchise?_No']==1].index, inplace=True)
dataset.head(30)

In [ ]:
#one hot encoding toepassen op de overige kolommen






#one-hot encoden van LIJST Features
datasets_encode_list = ['Han Solo','Luke Skywalker','Princess Leia Organa','Anakin Skywalker',
                        'Obi Wan Kenobi','Emperor Palpatine','Darth Vader','Lando Calrissian',
                       'Boba Fett','C-3P0','Jar Jar Binks','Padme Amidala','Yoda',
                        'Are you familiar with the Expanded Universe?',
                        'Do you consider yourself to be a fan of the Expanded Universe?','Gender','Education',
                        'phantom menace seen','attack of the clones seen','revenge of the sith seen','a new hope seen',
                        'empire strikes back seen','return of the jedi seen','Location','StarWars fan','R2 D2',
                        'Which character shot first?','Age'
                       ]

oude_datasets = dataset
for column in datasets_encode_list:
    dataset = pd.concat([dataset,pd.get_dummies(dataset[column],prefix=column)],axis=1)
    oude_datasets = dataset.drop([column],axis=1, inplace=True)
    

# # Han Solo omzetten
# dataset = pd.concat([dataset,pd.get_dummies(dataset['Han Solo'], prefix='Han Solo')],axis=1)
# #originele kolom verwijderen
# dataset.drop(['Han Solo'],axis=1, inplace=True)








dataset.head(25)



In [ ]:
# gebalanceerdheid controleren
sns.countplot(data=dataset, x="StarWars fan_Yes")
plt.title('StarWars fan')

In [ ]:
# Opsplitsen in features en targets
y = dataset['StarWars fan_Yes'].values
X = dataset.drop(['StarWars fan_Yes'], axis = 1).values

print(X.shape)
print(y.shape)


#1 lijst met 600 waarden, 130 features

In [ ]:
# Opsplitsen/opdelen in training set en test set (hier minstens 200 training sample)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.34, random_state=0)

#training set
print(X_train.shape)

#test set
print(X_test.shape)



In [ ]:
# Normaliseren / scaling van de training set en de test set
scaler = StandardScaler() #scalen
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train[1]

In [ ]:
# Train een logistic regression classifier

logreg = linear_model.LogisticRegression(C=1e2, solver='lbfgs') # C= Inverse of regularization strength;
                                                # must be a positive float. Like in support vector machines,
                                                # smaller values specify stronger regularization.
logreg.fit(X, y)

print('coefficiënten: ',logreg.coef_)
print('intercept:',logreg.intercept_)

In [ ]:
# testen van de logistic regression classifier

y_pred = logreg.predict(X_test)

print(classification_report(y_test,y_pred))

print(accuracy_score(y_test,y_pred)*100)

print(confusion_matrix(y_test,y_pred))

In [ ]:
# ROC

import sklearn.metrics as metrics
# calculate the fpr and tpr for all thresholds of the classification
probs = logreg.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#probability prediciton van ROC
print(probs)



In [ ]:
# Toevoegen van extra features
# Modeloptimalisatie en Hyperparameter tuning
# Automatisch toevoegen van hogere orde features

from sklearn.preprocessing import PolynomialFeatures


# Aanmaken van de hogere orde features
graad = 1

poly = PolynomialFeatures(graad)
poly.fit(X_train)
X_train_poly = poly.transform(X_train)
X_test_poly = poly.transform(X_test)
print('dimensie van X_train_poly: ',X_train_poly.shape)
print('dimensie van X_test_poly: ',X_test_poly.shape)

#scalen (optioneel?)
scaler_poly = StandardScaler()
scaler_poly.fit(X_train_poly)

X_train_poly = scaler_poly.transform(X_train_poly)
X_test_poly = scaler_poly.transform(X_test_poly)


# met L2 regularisatie via Ridge regression
lregmodel_poly = Ridge(alpha=20,tol=0.0001,fit_intercept=True)
lregmodel_poly.fit(X_train_poly,y_train)

print('R2 score op test set via L2: ',lregmodel_poly.score(X_test_poly,y_test))
# R2 -score via L2 op de trainingset
print('R2 score op training set via L2: ',lregmodel_poly.score(X_train_poly,y_train))


# met L1 regularisatie via Lasso regression
lregmodel_poly = Lasso(alpha=0.001,tol=0.001,fit_intercept=True)
lregmodel_poly.fit(X_train_poly,y_train)      
  
      
print('R2 score op test set via L1: ',lregmodel_poly.score(X_test_poly,y_test))
  
# R2 -score via L1 op de trainingset
print('R2 score op training set via L1: ',lregmodel_poly.score(X_train_poly,y_train)) 

In [ ]:
#Op beide trainingsets 0.99 en 0.99
#AUC = 1.00 wat betekenet dat men perfect de klassen kan onderscheiden
#Bij het wijzigen van C-parameter schommeling van ~0.1 accuracy

## Opdracht 4. Menselijke activiteit


De dataset 'activiteit.csv' bevat meetgegevens die van 30 personen via een smartphone werden afgenomen. Deze meetwaarden zijn onderandere afkomstig van een accelerometer en een gyroscoop die in de smartphone zijn ingebouwd.
De bedoeling is om om basis van deze meetwaarden de activiteit van de persoon zo nauwkeurig mogelijk te kunnen inschatten.
Er zijn 6 verschillende klasses.


Bepaal de 6 verschillende klasses en onderzoek meteen of de dataset gebalanceerd is (gelijkmatige verdeling van de verschillende klasses).
Bouw een logistic regression model dat zo accuraat mogelijk de activiteit van de persoon kan bepalen. Hierbij pas je alle nodige preprocessing stappen toe je nodig acht.
Het is ook toegestaann om features bij te maken of weg te laten. 

Schrijf jouw conclusies op met betrekking tot het finaal bekomen model.




In [ ]:
dataset = pd.read_csv('activiteit.csv')
dataset.head()

In [ ]:
# Uitwerking opdracht menselijke activiteit
sns.countplot(data=dataset,x='Activity')
plt.title('Countplot activity')


In [ ]:
# Lineaire correlatie tussen de features - heatmap

#dataset.corr()

# Visualiseer de onderlinge correlatiecoëfficiënten
#f, ax = plt.subplots(figsize=(12, 10))
#corr = dataset.corr()
#sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),
#            square=True, ax=ax,annot=True)

In [ ]:
#dataset.info()

In [ ]:
# statistische kerngetallen
#dataset.describe()


In [ ]:
#One-hot-encoding toepassen omdat ik strings moet omzetten naar getallen

#Species omzetten
dataset = pd.concat([dataset,pd.get_dummies(dataset['Activity'], prefix='Activity')],axis=1)
dataset.drop(['Activity'],axis=1, inplace=True)

dataset.head()

In [ ]:
# Opsporen van ontbrekende waarden

missing_values_count = dataset.isnull().sum()
print(missing_values_count)
msno.matrix(dataset)

In [ ]:
# Opsplitsen in features en targets
y = dataset['Activity_STANDING'].values
X = dataset.drop(['Activity_STANDING'], axis = 1).values

print(X.shape)
print(y.shape)


#1 lijst met 303 waarden, 13 features

In [ ]:
# Opsplitsen/opdelen in training set en test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#training set
print(X_train.shape)

#test set
print(X_test.shape)



In [ ]:
# Normaliseren / scaling van de training set en de test set
scaler = StandardScaler() #scalen
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train[1]

In [ ]:
# Train een logistic regression classifier

logreg = linear_model.LogisticRegression(C=1e3, solver='lbfgs') # C= Inverse of regularization strength;
                                                # must be a positive float. Like in support vector machines,
                                                # smaller values specify stronger regularization.
logreg.fit(X, y)

print('coefficiënten: ',logreg.coef_)
print('intercept:',logreg.intercept_)

In [ ]:
# testen van de logistic regression classifier

y_pred = logreg.predict(X_test)

print(classification_report(y_test,y_pred))

print(accuracy_score(y_test,y_pred)*100)

print(confusion_matrix(y_test,y_pred))

In [ ]:
# ROC

import sklearn.metrics as metrics
# calculate the fpr and tpr for all thresholds of the classification
probs = logreg.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
#probability prediciton van ROC
print(probs)

In [ ]:
# Toevoegen van extra features
# Modeloptimalisatie en Hyperparameter tuning
# Automatisch toevoegen van hogere orde features

from sklearn.preprocessing import PolynomialFeatures


# Aanmaken van de hogere orde features
graad = 1

poly = PolynomialFeatures(graad)
poly.fit(X_train)
X_train_poly = poly.transform(X_train)
X_test_poly = poly.transform(X_test)
print('dimensie van X_train_poly: ',X_train_poly.shape)
print('dimensie van X_test_poly: ',X_test_poly.shape)

#scalen (optioneel?)
scaler_poly = StandardScaler()
scaler_poly.fit(X_train_poly)

X_train_poly = scaler_poly.transform(X_train_poly)
X_test_poly = scaler_poly.transform(X_test_poly)


# met L2 regularisatie via Ridge regression
lregmodel_poly = Ridge(alpha=20,tol=0.01,fit_intercept=True)
lregmodel_poly.fit(X_train_poly,y_train)

print('R2 score op test set via L2: ',lregmodel_poly.score(X_test_poly,y_test))
# R2 -score via L2 op de trainingset
print('R2 score op training set via L2: ',lregmodel_poly.score(X_train_poly,y_train))


# met L1 regularisatie via Lasso regression
lregmodel_poly = Lasso(alpha=0.1,tol=0.01,fit_intercept=True)
lregmodel_poly.fit(X_train_poly,y_train)      
  
      
print('R2 score op test set via L1: ',lregmodel_poly.score(X_test_poly,y_test))
  
# R2 -score via L1 op de trainingset
print('R2 score op training set via L1: ',lregmodel_poly.score(X_train_poly,y_train)) 

In [ ]:
#R2 score op trainingsets 0.99 en 0.30(?)
#Bij het wijzigen van C-parameter enkel verschil van accuracy (~0.1)